# Team Project - Kickstarter Project

* Author: Julia Hammerer, Vanessa Mai
* Last Update: 15.06.2018



## Project Description
In this project we will look at a list of Crowdfunding Projects pulled from the Kickstarter website in 2018. Analysis will be mainly Data Explorations on the Project and may include (but not exclusively)
* compare successful and failed projects per country, and category
* look at size/funding amount of projects
* what's the value of successful projects, are there differences to failed ones?
* time series analysis 
* how much do people donate to projects in average

Here we will import all packages required for our analysis

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

## First Look on the data and data cleansing

Then first of all we import our csv file as a pandas dataframe


In [ ]:
ks_raw=pd.read_csv("ks-projects-201801.csv")

In [ ]:
ks_raw

When looking at the data we see that there are several fields with amounts (=pledged). The fields "pledged" and "goal" are in the original currency. There is also "usd pledged", usd_pledged_real", "uds_goal_real". "usd pledged" is the amount converted to us-dollar by Kickstarter. According to the description from Kaggle "usd_pledged_real" and "usd_goal_real" are converted using fixer.io by tonyplaysguitar. 
We'll use these fields, as this also have the "goal" in US-Dollars and we remove the other columns


In [ ]:
# remove unused columns, we don't need name and id, and the mentioned amounts
# we also do not need currency, as we have everything in US-Dollar
ks=ks_raw.drop(["ID", "name", "goal", "pledged", "usd pledged", "currency"], axis=1)

# we take a look at the datatypes, to look if we need to convert any fields to the appropriate data type
ks.dtypes 

We see that all the fields, that we need in a numeric form, are already automatically detected by python as numeric. 
However, to use the dates correctly, we need to convert launched and deadline into a datetime-datatype

In [ ]:
ks["launched"] = pd.to_datetime(ks["launched"], infer_datetime_format=True)
ks["deadline"] = pd.to_datetime(ks["deadline"], infer_datetime_format=True)

In [ ]:
# check for open projects
closed=ks["deadline"]>ks["launched"]
closed.value_counts()

We see that we only have closed projects here. As a prospect, we could include open projects and do a prediction on whether a project will be successful or not. Next step is to look at the data more closely and see some basic information on the data. For this we use the package pandas-profiling. See documenation [here](https://github.com/pandas-profiling/pandas-profiling)


In [ ]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(ks)

In [ ]:
# We also look at some other descriptive statistics
ks.describe()

So we have a basic summary first. The data seems to be complete, there are no missing values, so we do not have to deal with that. However, if we look closely at **"launched"**, which is the launch-day of a product, we can see that there are some in 1970. These might be missing values, so we have to think about what to do with that. As we have plenty records (~380 000) and the number of those observations are relatively low, it is enough if we just remove those observations

Another thing is, that the cardinality of **"category"** is very high, meaning there are many different values. If in the end, the number of observations per category is too small, it might not make much sense to explore them further. We will elaborate on that later more closely.

The **countries** are only displayed with their country code. We need to convert them to proper country names for better readability, later on also possibly some geocoding, and thus using maps for visualizations and such things. We will also elaborate on that later.

Pandas_Profiling also provides a really quick **Correlation Analysis**. We can see a high positive correlation between backers und pledged amount, which makes perfect sense, so this does not give us particularly interesting information. In fact, it is more interesting to see the average amount that backers give for the projects. Thus, we'll look into this later.

In [ ]:
## remove the rows with launched in 1970, as these are default values for missing entries
ks = ks[(ks["launched"].dt.year > 1970)]

We need to map the country code to the proper country names for better readability. Thus, we're using a file that includes the 2-digits code and the country name and we use this for mapping (file found [here](https://datahub.io/core/country-list#resource-data))

In [ ]:
# read in a mapping file for countries
country_mapping=pd.read_csv("country.csv")

In [ ]:
country_mapping

In [ ]:
# we'll do a left outer join, with our Kickstarter-Projectlist being left
ks= pd.merge(ks,country_mapping, how="left", left_on="country", right_on="Code" )

# we can see that the names are now properly written out
# but we'll need to remove the joining columns and rename the Country column from "Name" to "Country"
ks= ks.drop(["country", "Code"], axis=1)
ks=ks.rename(columns={"Name": "Country"})
ks.tail()

In [ ]:
# let's have a look at which countries we have:
print(ks.Country.unique())

We can see, that there is also "nan", so we had values which could not be mapped. We also looked into the raw data and saw the value "N,0"". This seems like a data quality issue. We can either remove the data or leave it in with the country value missing. 
We'll look into how many there are, if it is not too much, we can just remove it:

In [ ]:
# How many are "missing" in Country
print((ks.Country.isna()).sum())

3797 out of more than 300 000 is not much, so we can remove them without much concern.

In [ ]:
# remove of nan rows in Country
ks = ks[(ks["Country"].notna())]

## Distributions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Distribution of Project Status
First we take a look at the distribution of the status of the Projects


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
sns.countplot(x="state",data=ks, ax=ax,palette="Paired")


In this plot the number of projects and their respective status are displayed.
We can see that most Projects have failed. The values "live", "undefined" and "suspended" have the least values and their status is not clear, there is no proper description on the source of the data (Kaggle) either. So these values will be removed in the next step. We concentrate on the more meaningful values "failed", "canceled" and "successful"
Next we analyse the rate of success.

In [ ]:
# Removing Rows
ks= ks[ks.state != "live"]
ks= ks[ks.state != "undefined"]
ks= ks[ks.state != "suspended"]

In [ ]:
# Success rate
success_rate = (ks.state == 'successful').sum()/len(ks.index)
print(success_rate)

Rougly 36% of all Projects, this dataset are won. 
As one of the next steps, we try to find out differences between failed and cancelled.

### Distribution of the Main Category

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="main_category",data=ks, ax=ax, order = ks['main_category'].value_counts().index)

We have a deeper look at the distribution of categories and main categories

In [ ]:
ks.groupby("main_category")["category"].value_counts()

 How is the distribution of failed,canceld or successful Projects in each category?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="main_category",hue="state",data=ks, ax=ax, palette="Paired")

How many entries in the dataset are from which country?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(y="Country",data=ks,order = ks['Country'].value_counts().index,ax=ax)

### Distribution of the Project state across the countries

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(y="Country",hue="state",data=ks, ax=ax, palette="Paired")

### How many backers do the projects have in each country?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.barplot(y="Country",x="backers",data=ks,ci=None, ax=ax)
plt.show()

In [ ]:
print("Minimal goal value:", ks["usd_goal_real"].min())
print("Mean goal value:", ks["usd_goal_real"].mean())
print("Maximum goal value:", ks["usd_goal_real"].max())

## Project Sizes
After having a look at various counts of the Projects, we need to see the sizes of those projects. 
e.g. What are the goals of the projects in various categories?


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
plt.yscale("log")
# plt.xscale("log")

plt.boxplot(ks.usd_goal_real)
# sns.distplot(x, kde=False, rug=True);

In the first histogram showing the goal of the projects in the Profile Report, we don't see much. We only saw, that the USD_goal is highly skewed. Here, we produced a boxplot with a logarithmic scale to see the data better. The biggest fraction of all projects' goals are between **1000** and **100000**. Let's create another histogram for those projects to see the distribution within that fraction better. 

In [ ]:
ks_mostCommon = ks[((ks["usd_goal_real"]<100000) & (ks["usd_goal_real"]>1000)) ]
print("Projects between 1000 - 1000000 as a goal: ",len(ks_small.index))
print("Total Projects: ",len(ks.index))


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
# plt.yscale("log")
# plt.xscale("log")
plt.rcParams["patch.force_edgecolor"] = True


sns.distplot(ks_small.usd_goal_real, bins=60, kde = False, hist_kws=dict(edgecolor="black"))
# sns.distplot(x, kde=False, rug=True);

plt.show()

### Project Sizes per Main Category
We want to also see if there are bigger differences between the different main categories. As, the categories have a very high cardinality, we don't look at it at this point.

In [ ]:
plt.figure(figsize=(20,20))
plt.yscale("log")
sns.boxplot(x="main_category", y="usd_goal_real",data=ks)


In this plot, we can see more clearly the outliers of the main categories, which are existing in any main category. In general, we see that the range stays in around the same amount, with only "Technology" slightly sticking out for having higher goals. In "Publishing" and "Art" there are also projects with a very low goal. In summary, "Publishing", "Crafts" and "Art" have slightly lower goals, whereas "Technology" requires most donations. As the other plots, this is also on a logarithmic scale. 



### Project Sizes per Country
Also, interesting would be to see if there are differences by counties:


In [ ]:
plt.figure(figsize=(20,20))
plt.yscale("log")
sns.boxplot(x="Country", y="usd_goal_real",data=ks)


There isn't much variation between the countries, except for US having some particularly low goals for some project. This isn't surprising though, as most projects come from USA.

We want to see the average, median, and possibly other basic statistics for each category and country.

In [ ]:
ks[["main_category", "usd_goal_real"]].groupby("main_category").describe().sort_values(by= ('usd_goal_real', 'mean'), ascending =False)

In [ ]:
ks[["Country", "usd_goal_real"]].groupby("Country").describe().sort_values(by= ('usd_goal_real', 'mean'), ascending =False)

## Success Chances and Rates

After seeing the distributions and values of the projects and having a feeling about the data, we want to see, what the success rates are and see if any attributes have any significant correlation with the outcome of the project. In the following, we will conduct some univariate analysis.

First we want to add some more interesting values
* Duration of the project: How much time between launch-day and deadline in days?
* Ratio of Duration and goal: what would be the goal per day?
* Delta pledged: how much more than the goal has been pledged or how much is missing?
* Average pledged per backer: how much in average do people donate per project?
* pledged rate: How much per day in average?

In [ ]:
ks["project_length"]=(ks["deadline"]-ks["launched"])/np.timedelta64(1, 'D')
ks["goal_per_day"]= ks["usd_goal_real"]/ks["project_length"]
ks["pledged_delta"]= ks["usd_pledged_real"]-ks["usd_goal_real"]
ks["pledged_per_backer"]=ks["usd_pledged_real"]/ks["backers"]
ks["pledged_per_day"]= ks["project_length"]/ks["usd_pledged_real"]

In [ ]:
ks.describe()

In [ ]:
ks.head()

First, we look into the projects goal again. 
This time with regard to the project status.

In [ ]:
# plot project sizes with regard to project status
# use log-scale again, for the highly skewed data
plt.figure(figsize=(15,15))
plt.yscale("log")

sns.boxplot(x="state", y="usd_goal_real",data=ks)


We don't see a huge difference between the different statuses. Successful projects seem to have slightly smaller goals and much less and lower outliers.
It looks like projects with a goal up to 100 000 USD, have a more realistic chance of being successful. We also look into this including the categories.

In [ ]:
plt.figure(figsize=(20,20))
plt.yscale("log")
sns.set(style="ticks")
sns.boxplot(x="main_category", y="usd_goal_real", hue="state",data=ks, palette="Paired")
# sns.despine(offset=10, trim=True)


In general, the successful projects in any category are more shifted towards having a lower goal. Expecially in "Crafts" and in "Technology" it can be nicely observerd. At this point we should also look at the relative successrate per category:

In [ ]:
# Success rate per category
mainCategory_byState = ks.groupby('main_category').state.value_counts().unstack()
success_rate_perCategory= mainCategory_byState.successful / (mainCategory_byState.successful + mainCategory_byState.canceled + mainCategory_byState.failed)
success_rate_perCategory=success_rate_perCategory.sort_values(ascending=True)
plt.figure(figsize=(20,10))

success_rate_perCategory.plot(kind="barh")

Technologies are less successful, because they require high donations. Crafts and Journalism are also less successful, and also their goal tend to be higher. Here Particularly you can see the difference in the sizes.

Next we look into the distribution of the project length. 

In [ ]:
plt.figure(figsize=(15,15))
sns.distplot(ks.project_length, bins=25, kde = False, hist_kws=dict(edgecolor="black"))

We see a nicely shaped distribution, with most projects having a length of approximately 30 days/ 1 month. There is another smaller peak at 60 days/ 2 months.

In [ ]:
plt.figure(figsize=(20,20))
sns.boxplot(x="main_category", y="project_length",data=ks)


Next we look into how the project length vary for successful, cancelled and failed projects.

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(x="state", y="project_length",data=ks)


Even though the median looks the same for all three states, successful projects seems to have shorter deadlines. They are less skewed, even though there are still many outliers. 

We'll look into the different categories again:

In [ ]:
plt.figure(figsize=(20,20))
sns.set(style="ticks")
sns.boxplot(x="main_category", y="project_length", hue="state",data=ks, palette="Paired")
sns.despine(offset=10, trim=True)


We see the duration is shorter for successful projects in all categories.

Next we look into the goal per day, which is the ratio between the duration and the total goal. We look into that with regard to the project outcome and also include the categories.


In [ ]:
ks[["state", "goal_per_day"]].groupby("state").describe()

We can see, that the maximum goal_per_day for successful projects is 68369. For cancelled and failed projects it is much higher!! Let's look at the distribution of  projects with goal_per_day < 70000

In [ ]:
# Success rate per category
test = ks[(ks["goal_per_day"]>70000)].groupby('main_category').state.value_counts().unstack()
print(test)
# v= test.successful / (test.successful + test.canceled + test.failed)
# v=v.sort_values(ascending=True)
# v

In [ ]:
plt.figure(figsize=(20,20))
plt.yscale("log")
sns.boxplot(x="main_category", y="goal_per_day",data=ks[(ks["goal_per_day"]<70000)], palette="Paired",  hue="state")

## Temporal Analysis

In [ ]:
describe()